# 2nd approach to Facial Emotion Recognition


In [1]:
import numpy as np
import cv2 as cv
import pandas as pd
import dlib
from imutils import face_utils
from sklearn.preprocessing import normalize
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from scipy.spatial.distance import pdist
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import History
from keras import optimizers
from keras.callbacks import Callback
from keras import regularizers
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import matthews_corrcoef
import matplotlib.pyplot as plt

C:\Users\Manvir\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Preproccessing

First the dataset will be loaded.

In [2]:
x = [0]*980
y = [0]*980
from os import listdir
path = 'KDEF_and_AKDEF/KDEF'
folders = listdir(path)
c = 0
for f in folders:
    files = listdir(path + '/'+ f)
    for fi in files:
        if fi[6] == 'S':
            x[c] = cv.imread((path + '/' + f + '/' + fi))
            #print(path + '/' + f + '/' + fi)
            y[c] = fi[4:6]
            c += 1

In [3]:
len(x)

980

In [4]:
y = np.array(y)

Now 68 points on each face will be located.

In [5]:
# initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor
p = "shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(p)

In [6]:
def preproc(x):
    desf = [0]*len(x)
    c = 0
    for image in x:
        
        # A 3x3 Gaussian Blur will now be applied
        image = cv.GaussianBlur(image, (3,3), 0)
        
        # Now need to apply a Contrast Adaptive Histogran Equalization
        image2 = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
        clahe = cv.createCLAHE(clipLimit=4, tileGridSize=(8,8))
        cl1 = clahe.apply(image2.copy())
        
        # detect faces in the grayscale image
        rects = detector(cl1, 0)
        
        # loop over the face detections
        for (i, rect) in enumerate(rects):
            # determine the facial landmarks for the face region, then
            # convert the facial landmark (x, y)-coordinates to a NumPy
            # array
            shape = predictor(cl1, rect)
            shape = face_utils.shape_to_np(shape)
            
        desf[c] = shape
        c += 1
    return(desf)

In [7]:
desf = preproc(x)

In [8]:
desf = np.array(desf)

Now the neutral face class will be removed after the displacement from the nueral faces are determined. 

In [9]:
res = [0]*980
def feat(v1, v2):
    result = [0]*len(v1)
    for i in range(len(v1)):
        result[i] = np.linalg.norm(v1[i]-v2[i])
    result = np.array(result)
    return result
for i in range(0, len(desf), 7):
    res[i] = feat(desf[i], desf[i+4])
    res[i+1] = feat(desf[i+1], desf[i+4])
    res[i+2] = feat(desf[i+2], desf[i+4])
    res[i+3] = feat(desf[i+3], desf[i+4])
    res[i+4] = np.array([0]*len(desf[0]))
    res[i+5] = feat(desf[i+5], desf[i+4])
    res[i+6] = feat(desf[i+6], desf[i+4])
res = np.array(res)
neutral_indices = y!='NE'
y = y[neutral_indices]
res = res[neutral_indices]

In [10]:
X_train, X_test, y_train, y_test = train_test_split(res, y, test_size=0.1, random_state=42)

In [11]:
len(X_train)

756

In [12]:
len(y_train)

756

In [13]:
X_train

array([[ 3.60555128,  2.82842712,  2.23606798, ...,  7.81024968,
         4.12310563,  6.70820393],
       [ 7.        ,  8.06225775, 10.63014581, ...,  8.48528137,
         6.        ,  5.        ],
       [22.13594362, 17.2626765 , 12.04159458, ..., 10.63014581,
         8.        , 10.        ],
       ...,
       [ 8.60232527,  7.81024968,  9.8488578 , ...,  8.        ,
         7.28010989,  8.60232527],
       [ 2.23606798,  3.16227766,  3.16227766, ...,  8.94427191,
         9.05538514,  9.21954446],
       [ 9.21954446,  7.61577311,  6.32455532, ..., 38.20994635,
        37.12142239, 37.12142239]])

In [14]:
le = LabelEncoder()
le.fit(y_train)

LabelEncoder()

In [15]:
le.classes_

array(['AF', 'AN', 'DI', 'HA', 'SA', 'SU'], dtype='<U2')

In [16]:
len(le.classes_)

6

In [17]:
y_actual = le.transform(y_train) 

In [18]:
y_actual2 = le.transform(y_test) 

The labels for the train and test sets will onehot encoded for the neural networks.

In [19]:
onehot_encoder = OneHotEncoder(categories='auto')

In [20]:
onehot_encoder = onehot_encoder.fit(y_actual.reshape(X_train.shape[0], 1))

In [21]:
y1 = onehot_encoder.transform(y_actual.reshape(X_train.shape[0], 1))

In [22]:
y2 = onehot_encoder.transform(y_actual2.reshape(X_test.shape[0], 1))

X_train will now be normalized

In [23]:
scaler = StandardScaler().fit(X_train)

In [24]:
scaler

StandardScaler(copy=True, with_mean=True, with_std=True)

In [25]:
scaler.mean_

array([13.4212468 , 12.53215287, 12.30133646, 12.36494283, 12.46424551,
       12.39781892, 12.75165178, 13.42837621, 13.94364526, 13.50389551,
       12.96401529, 12.5197699 , 12.4377812 , 12.0878501 , 11.8222716 ,
       12.0450053 , 13.08815201,  9.5147746 , 10.42083818, 11.16663643,
       11.36994764, 10.85015242, 11.14593426, 11.60185707, 11.31126404,
       10.19663181,  8.97577403,  5.00996417,  5.48640228,  6.98009636,
        8.84971507,  7.37795933,  6.65083427,  6.50755377,  6.73105837,
        7.65627738,  5.10428545,  5.189194  ,  4.61789274,  4.2478041 ,
        5.60133266,  6.13196626,  4.40580307,  4.77568802,  5.38316336,
        5.31268569,  6.15516776,  5.75065462, 14.92970013, 12.59719201,
       10.21048503, 10.20861452, 10.47567181, 12.90346069, 15.23270426,
       15.91684216, 16.85619951, 16.66561146, 16.64496189, 15.37643615,
       14.5995465 , 11.29173651, 10.92762198, 11.59126954, 15.05452864,
       15.63251388, 15.23017383, 15.35196299])

In [26]:
scaler.scale_

array([ 9.57798826,  8.87140114,  8.58138009,  8.483784  ,  8.66886368,
        8.96320588,  9.7506549 , 10.45196283, 10.80206208, 10.31876042,
        9.55721676,  8.8756141 ,  8.63547567,  8.36384994,  8.30621174,
        8.55637178,  9.18385181,  6.74572299,  6.96115787,  7.7875656 ,
        8.16784602,  7.88276278,  7.69437032,  7.98701541,  7.64678332,
        6.82478143,  6.42490435,  5.3921111 ,  5.75812503,  6.68621743,
        7.88969949,  6.17081385,  6.08541507,  6.26949007,  6.1176513 ,
        6.22413707,  5.18383988,  5.16649165,  5.01338821,  4.9151339 ,
        5.18054393,  5.36333776,  4.94215319,  5.08808128,  5.23966525,
        5.21383745,  5.47675654,  5.2897598 , 11.23714226,  9.3371541 ,
        7.67089019,  7.6233982 ,  7.61375695,  9.15370879, 11.06201428,
       11.69246165, 14.53970184, 14.91051751, 14.30329097, 11.3191613 ,
       10.78567362,  8.37556718,  8.07019721,  8.35819368, 10.72923382,
       12.97102639, 13.32467996, 12.84733455])

In [27]:
X_train = scaler.transform(X_train)

In [28]:
X_test = scaler.transform(X_test)

## SVM classifiers

The best hyperparameters for a support vector machine with the 'rbf' kernel will now be determined.

In [25]:
clf = SVC(kernel='rbf')

params = {
    'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1, 10, 100],
    'C': [0.001, 0.01, 0.1, 1]
}

gs = RandomizedSearchCV(clf, params, refit=False, cv=10, scoring='accuracy', verbose=3, random_state=42) #verbose=2

gs.fit(X_train, y_actual)
print(gs.best_score_, gs.best_params_)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] gamma=1, C=1 ....................................................
[CV] .......... gamma=1, C=1, score=0.16666666666666666, total=   0.0s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] gamma=1, C=1 ....................................................
[CV] ........... gamma=1, C=1, score=0.1794871794871795, total=   0.0s


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s


[CV] gamma=1, C=1 ....................................................
[CV] .......... gamma=1, C=1, score=0.16883116883116883, total=   0.0s
[CV] gamma=1, C=1 ....................................................
[CV] .......... gamma=1, C=1, score=0.16883116883116883, total=   0.0s
[CV] gamma=1, C=1 ....................................................
[CV] .......... gamma=1, C=1, score=0.16883116883116883, total=   0.0s
[CV] gamma=1, C=1 ....................................................
[CV] .......... gamma=1, C=1, score=0.17105263157894737, total=   0.0s
[CV] gamma=1, C=1 ....................................................
[CV] .......... gamma=1, C=1, score=0.17567567567567569, total=   0.0s
[CV] gamma=1, C=1 ....................................................
[CV] .......... gamma=1, C=1, score=0.17567567567567569, total=   0.0s
[CV] gamma=1, C=1 ....................................................
[CV] ........... gamma=1, C=1, score=0.2054794520547945, total=   0.0s
[CV] g

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   12.7s finished
C:\Users\Manvir\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


0.6997354497354498 {'gamma': 'scale', 'C': 1}


In [36]:
clf = SVC(kernel='rbf', gamma='scale', C=1)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)

The accuracy will now be determined.

In [37]:
accuracy_score(y_test, pred)

0.6666666666666666

In [27]:
confusion_matrix(y_test, pred, labels=le.classes_)

array([[ 8,  2,  0,  0,  2,  2],
       [ 0,  8,  1,  0,  2,  0],
       [ 1,  1,  9,  0,  3,  1],
       [ 0,  0,  0, 11,  1,  0],
       [ 1,  4,  0,  2,  6,  1],
       [ 3,  1,  0,  0,  0, 14]], dtype=int64)

In [28]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

          AF       0.62      0.57      0.59        14
          AN       0.50      0.73      0.59        11
          DI       0.90      0.60      0.72        15
          HA       0.85      0.92      0.88        12
          SA       0.43      0.43      0.43        14
          SU       0.78      0.78      0.78        18

   micro avg       0.67      0.67      0.67        84
   macro avg       0.68      0.67      0.67        84
weighted avg       0.69      0.67      0.67        84



The Matthew's correlation coefficient will now be determined.

In [29]:
matthews_corrcoef(y_test, pred)

0.6021212493973955

The best hyperparameters for a support vector machine with the 'poly' kernel will now be determined.

In [30]:
clf = SVC(kernel='poly')

params = {
    'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1, 10, 100],
    'degree': [0, 1, 2, 3, 4, 5, 6],
    'C': [0.001, 0.01, 0.1, 1]
}

gs = RandomizedSearchCV(clf, params, refit=False, cv=10, scoring='accuracy', verbose=3, random_state=42) #verbose=2

gs.fit(X_train, y_actual)
print(gs.best_score_, gs.best_params_)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] gamma=auto, degree=1, C=0.001 ...................................
[CV]  gamma=auto, degree=1, C=0.001, score=0.16666666666666666, total=   0.0s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] gamma=auto, degree=1, C=0.001 ...................................
[CV]  gamma=auto, degree=1, C=0.001, score=0.16666666666666666, total=   0.0s


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s


[CV] gamma=auto, degree=1, C=0.001 ...................................
[CV]  gamma=auto, degree=1, C=0.001, score=0.16883116883116883, total=   0.0s
[CV] gamma=auto, degree=1, C=0.001 ...................................
[CV]  gamma=auto, degree=1, C=0.001, score=0.16883116883116883, total=   0.0s
[CV] gamma=auto, degree=1, C=0.001 ...................................
[CV]  gamma=auto, degree=1, C=0.001, score=0.16883116883116883, total=   0.0s
[CV] gamma=auto, degree=1, C=0.001 ...................................
[CV]  gamma=auto, degree=1, C=0.001, score=0.17105263157894737, total=   0.0s
[CV] gamma=auto, degree=1, C=0.001 ...................................
[CV]  gamma=auto, degree=1, C=0.001, score=0.17567567567567569, total=   0.0s
[CV] gamma=auto, degree=1, C=0.001 ...................................
[CV]  gamma=auto, degree=1, C=0.001, score=0.17567567567567569, total=   0.0s
[CV] gamma=auto, degree=1, C=0.001 ...................................
[CV]  gamma=auto, degree=1, C=0.001

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    8.6s finished
C:\Users\Manvir\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


0.6838624338624338 {'gamma': 'auto', 'degree': 1, 'C': 1}


In [38]:
clf = SVC(kernel='poly', gamma='auto', degree=1, C=1)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)

The accuracy will now be determined.

In [39]:
accuracy_score(y_test, pred)

0.6785714285714286

In [32]:
confusion_matrix(y_test, pred, labels=le.classes_)

array([[ 7,  2,  0,  0,  3,  2],
       [ 1,  8,  0,  0,  2,  0],
       [ 2,  1,  9,  0,  3,  0],
       [ 0,  0,  0, 11,  1,  0],
       [ 1,  2,  0,  2,  8,  1],
       [ 2,  1,  0,  0,  1, 14]], dtype=int64)

In [33]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

          AF       0.54      0.50      0.52        14
          AN       0.57      0.73      0.64        11
          DI       1.00      0.60      0.75        15
          HA       0.85      0.92      0.88        12
          SA       0.44      0.57      0.50        14
          SU       0.82      0.78      0.80        18

   micro avg       0.68      0.68      0.68        84
   macro avg       0.70      0.68      0.68        84
weighted avg       0.71      0.68      0.68        84



The Matthew's correlation coefficient will now be determined.

In [34]:
matthews_corrcoef(y_test, pred)

0.6170588103401528

## Neural Networks

### One Hidden Layer

The best hyperparameters for a neural network with one hidden layer will now be determined.

In [31]:
# Function to create model, required for KerasClassifier
def create_model(optimizer='adam', init='glorot_uniform', h1= 10, dropout_rate=0.0, learning_rate=0.001):
    # create model
    model = Sequential()
    model.add(Dense(h1, input_dim=X_train.shape[1], kernel_initializer=init, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    model.add(Dense(len(le.classes_), kernel_initializer=init, activation='softmax'))
    # Compile model
    optimizer = optimizers.Adam(lr=learning_rate,  clipvalue=100)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy']) # 
    return model

# fix random seed for reproducibility
seed = 42
np.random.seed(seed)
# create model
model = KerasClassifier(build_fn=create_model, verbose=2)
# grid search epochs, batch size and optimizer
init = ['glorot_uniform', 'normal', 'uniform']
epochs = [50, 100, 150]
batches = [64, 128, 256]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
h1 = [5, 10, 20, 40, 50, 100, 150, 200, 300, 400, 500]
learning_rate= [0.0001, 0.005, 0.001, 0.01, 0.05, 0.1, 0.5, 1, 5]
param_grid = dict(epochs=epochs, batch_size=batches, dropout_rate=dropout_rate, init=init, h1=h1, learning_rate=learning_rate)
cvi = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid, verbose=4, cv=cvi, return_train_score=True)
grid_result = grid.fit(X_train, y_actual)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] learning_rate=0.001, init=glorot_uniform, h1=200, epochs=100, dropout_rate=0.6, batch_size=256 
Epoch 1/100
 - 0s - loss: 2.9956 - acc: 0.1947
Epoch 2/100
 - 0s - loss: 2.1743 - acc: 0.3422
Epoch 3/100
 - 0s - loss: 1.7709 - acc: 0.4204
Epoch 4/100
 - 0s - loss: 1.6933 - acc: 0.4543
Epoch 5/100
 - 0s - loss: 1.4777 - acc: 0.5206
Epoch 6/100
 - 0s - loss: 1.3832 - acc: 0.5605
Epoch 7/100
 - 0s - loss: 1.3942 - acc: 0.5442
Epoch 8/100
 - 0s - loss: 1.2730 - acc: 0.5811
Epoch 9/100
 - 0s - loss: 1.2617 - acc: 0.5826
Epoch 10/100
 - 0s - loss: 1.2090 - acc: 0.6283
Epoch 11/100
 - 0s - loss: 1.1061 - acc: 0.6268
Epoch 12/100
 - 0s - loss: 1.0415 - acc: 0.6490
Epoch 13/100
 - 0s - loss: 1.0144 - acc: 0.6652
Epoch 14/100
 - 0s - loss: 1.0374 - acc: 0.6460
Epoch 15/100
 - 0s - loss: 1.0807 - acc: 0.6490
Epoch 16/100
 - 0s - loss: 0.9752 - acc: 0.6622
Epoch 17/100
 - 0s - loss: 0.9859 - acc: 0.6917
Epoch 18/100
 - 0s - loss: 0.9369 - acc: 0.6829
Epoch 19/100
 - 0s - loss: 0.9660 - acc: 0.6

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.6s remaining:    0.0s


[CV] learning_rate=0.001, init=glorot_uniform, h1=200, epochs=100, dropout_rate=0.6, batch_size=256 
Epoch 1/100
 - 0s - loss: 2.9607 - acc: 0.1785
Epoch 2/100
 - 0s - loss: 2.3523 - acc: 0.3068
Epoch 3/100
 - 0s - loss: 1.8290 - acc: 0.3953
Epoch 4/100
 - 0s - loss: 1.6307 - acc: 0.4720
Epoch 5/100
 - 0s - loss: 1.4443 - acc: 0.5265
Epoch 6/100
 - 0s - loss: 1.3964 - acc: 0.5619
Epoch 7/100
 - 0s - loss: 1.4179 - acc: 0.5516
Epoch 8/100
 - 0s - loss: 1.2753 - acc: 0.5870
Epoch 9/100
 - 0s - loss: 1.2181 - acc: 0.5944
Epoch 10/100
 - 0s - loss: 1.1378 - acc: 0.6121
Epoch 11/100
 - 0s - loss: 1.1033 - acc: 0.6401
Epoch 12/100
 - 0s - loss: 1.1528 - acc: 0.6180
Epoch 13/100
 - 0s - loss: 1.0388 - acc: 0.6372
Epoch 14/100
 - 0s - loss: 0.9992 - acc: 0.6829
Epoch 15/100
 - 0s - loss: 1.0001 - acc: 0.6490
Epoch 16/100
 - 0s - loss: 0.9021 - acc: 0.6726
Epoch 17/100
 - 0s - loss: 0.9594 - acc: 0.6578
Epoch 18/100
 - 0s - loss: 0.9869 - acc: 0.6608
Epoch 19/100
 - 0s - loss: 0.9432 - acc: 0.6

...

Epoch 25/150
 - 0s - loss: 0.9962 - acc: 0.6833
Epoch 26/150
 - 0s - loss: 0.9918 - acc: 0.6789
Epoch 27/150
 - 0s - loss: 0.9833 - acc: 0.6584
Epoch 28/150
 - 0s - loss: 0.9914 - acc: 0.6598
Epoch 29/150
 - 0s - loss: 0.9437 - acc: 0.6965
Epoch 30/150
 - 0s - loss: 1.0018 - acc: 0.6745
Epoch 31/150
 - 0s - loss: 0.9880 - acc: 0.6950
Epoch 32/150
 - 0s - loss: 0.8912 - acc: 0.6979
Epoch 33/150
 - 0s - loss: 0.9238 - acc: 0.6774
Epoch 34/150
 - 0s - loss: 0.8497 - acc: 0.7023
Epoch 35/150
 - 0s - loss: 0.9056 - acc: 0.6965
Epoch 36/150
 - 0s - loss: 0.8456 - acc: 0.7038
Epoch 37/150
 - 0s - loss: 0.8256 - acc: 0.6921
Epoch 38/150
 - 0s - loss: 0.7788 - acc: 0.7346
Epoch 39/150
 - 0s - loss: 0.8080 - acc: 0.7243
Epoch 40/150
 - 0s - loss: 0.8354 - acc: 0.7214
Epoch 41/150
 - 0s - loss: 0.7531 - acc: 0.7449
Epoch 42/150
 - 0s - loss: 0.7615 - acc: 0.7170
Epoch 43/150
 - 0s - loss: 0.7783 - acc: 0.7361
Epoch 44/150
 - 0s - loss: 0.7968 - acc: 0.7155
Epoch 45/150
 - 0s - loss: 0.7496 - acc:

Epoch 40/150
 - 0s - loss: 0.8229 - acc: 0.7174
Epoch 41/150
 - 0s - loss: 0.8064 - acc: 0.7247
Epoch 42/150
 - 0s - loss: 0.8104 - acc: 0.7072
Epoch 43/150
 - 0s - loss: 0.7310 - acc: 0.7365
Epoch 44/150
 - 0s - loss: 0.7931 - acc: 0.7116
Epoch 45/150
 - 0s - loss: 0.7835 - acc: 0.7379
Epoch 46/150
 - 0s - loss: 0.7079 - acc: 0.7394
Epoch 47/150
 - 0s - loss: 0.7757 - acc: 0.7218
Epoch 48/150
 - 0s - loss: 0.7320 - acc: 0.7335
Epoch 49/150
 - 0s - loss: 0.7182 - acc: 0.7394
Epoch 50/150
 - 0s - loss: 0.6704 - acc: 0.7570
Epoch 51/150
 - 0s - loss: 0.7247 - acc: 0.7379
Epoch 52/150
 - 0s - loss: 0.7747 - acc: 0.7306
Epoch 53/150
 - 0s - loss: 0.7268 - acc: 0.7408
Epoch 54/150
 - 0s - loss: 0.7231 - acc: 0.7233
Epoch 55/150
 - 0s - loss: 0.7398 - acc: 0.7204
Epoch 56/150
 - 0s - loss: 0.7394 - acc: 0.7555
Epoch 57/150
 - 0s - loss: 0.6918 - acc: 0.7438
Epoch 58/150
 - 0s - loss: 0.7521 - acc: 0.7365
Epoch 59/150
 - 0s - loss: 0.6745 - acc: 0.7496
Epoch 60/150
 - 0s - loss: 0.6536 - acc:

Epoch 55/150
 - 0s - loss: 0.7337 - acc: 0.7222
Epoch 56/150
 - 0s - loss: 0.7394 - acc: 0.7398
Epoch 57/150
 - 0s - loss: 0.6925 - acc: 0.7515
Epoch 58/150
 - 0s - loss: 0.6469 - acc: 0.7690
Epoch 59/150
 - 0s - loss: 0.7025 - acc: 0.7500
Epoch 60/150
 - 0s - loss: 0.6977 - acc: 0.7383
Epoch 61/150
 - 0s - loss: 0.7266 - acc: 0.7251
Epoch 62/150
 - 0s - loss: 0.6604 - acc: 0.7325
Epoch 63/150
 - 0s - loss: 0.7279 - acc: 0.7339
Epoch 64/150
 - 0s - loss: 0.6602 - acc: 0.7602
Epoch 65/150
 - 0s - loss: 0.6885 - acc: 0.7412
Epoch 66/150
 - 0s - loss: 0.6547 - acc: 0.7588
Epoch 67/150
 - 0s - loss: 0.6659 - acc: 0.7558
Epoch 68/150
 - 0s - loss: 0.6133 - acc: 0.7544
Epoch 69/150
 - 0s - loss: 0.6237 - acc: 0.7558
Epoch 70/150
 - 0s - loss: 0.6311 - acc: 0.7719
Epoch 71/150
 - 0s - loss: 0.6155 - acc: 0.7690
Epoch 72/150
 - 0s - loss: 0.6545 - acc: 0.7675
Epoch 73/150
 - 0s - loss: 0.6403 - acc: 0.7646
Epoch 74/150
 - 0s - loss: 0.6195 - acc: 0.7602
Epoch 75/150
 - 0s - loss: 0.6056 - acc:

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed: 14.0min finished
C:\Users\Manvir\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Epoch 1/100
 - 5s - loss: 3.0364 - acc: 0.1601
Epoch 2/100
 - 0s - loss: 2.2816 - acc: 0.2804
Epoch 3/100
 - 0s - loss: 1.8415 - acc: 0.4048
Epoch 4/100
 - 0s - loss: 1.6197 - acc: 0.4854
Epoch 5/100
 - 0s - loss: 1.4607 - acc: 0.5238
Epoch 6/100
 - 0s - loss: 1.4372 - acc: 0.5556
Epoch 7/100
 - 0s - loss: 1.3736 - acc: 0.5847
Epoch 8/100
 - 0s - loss: 1.2521 - acc: 0.6098
Epoch 9/100
 - 0s - loss: 1.2310 - acc: 0.6164
Epoch 10/100
 - 0s - loss: 1.1474 - acc: 0.6283
Epoch 11/100
 - 0s - loss: 1.1690 - acc: 0.6111
Epoch 12/100
 - 0s - loss: 1.0444 - acc: 0.6376
Epoch 13/100
 - 0s - loss: 1.0535 - acc: 0.6534
Epoch 14/100
 - 0s - loss: 1.0425 - acc: 0.6362
Epoch 15/100
 - 0s - loss: 1.0464 - acc: 0.6362
Epoch 16/100
 - 0s - loss: 0.9660 - acc: 0.6667
Epoch 17/100
 - 0s - loss: 0.9092 - acc: 0.6772
Epoch 18/100
 - 0s - loss: 0.9450 - acc: 0.6640
Epoch 19/100
 - 0s - loss: 0.9336 - acc: 0.6693
Epoch 20/100
 - 0s - loss: 0.9229 - acc: 0.6720
Epoch 21/100
 - 0s - loss: 0.8777 - acc: 0.6984
E

In [32]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.720899 using {'learning_rate': 0.001, 'init': 'glorot_uniform', 'h1': 200, 'epochs': 100, 'dropout_rate': 0.6, 'batch_size': 256}
0.720899 (0.068808) with: {'learning_rate': 0.001, 'init': 'glorot_uniform', 'h1': 200, 'epochs': 100, 'dropout_rate': 0.6, 'batch_size': 256}
0.541005 (0.058968) with: {'learning_rate': 0.0001, 'init': 'glorot_uniform', 'h1': 20, 'epochs': 150, 'dropout_rate': 0.7, 'batch_size': 128}
0.674603 (0.066129) with: {'learning_rate': 0.1, 'init': 'uniform', 'h1': 400, 'epochs': 150, 'dropout_rate': 0.0, 'batch_size': 64}
0.206349 (0.049603) with: {'learning_rate': 5, 'init': 'normal', 'h1': 10, 'epochs': 50, 'dropout_rate': 0.6, 'batch_size': 64}
0.708995 (0.050407) with: {'learning_rate': 0.001, 'init': 'glorot_uniform', 'h1': 200, 'epochs': 50, 'dropout_rate': 0.4, 'batch_size': 256}
0.710317 (0.071710) with: {'learning_rate': 0.01, 'init': 'glorot_uniform', 'h1': 40, 'epochs': 100, 'dropout_rate': 0.3, 'batch_size': 128}
0.207672 (0.062577) with: {'lear

In [66]:
model = Sequential()
model.add(Dense(200, input_dim=X_train.shape[1], kernel_initializer='glorot_uniform', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.6))
model.add(Dense(len(le.classes_), kernel_initializer='glorot_uniform', activation='softmax'))
# Compile model
optimizer = optimizers.Adam(lr=0.001,  clipvalue=100)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

model.fit(X_train, y1, epochs = 100, batch_size = 256)

Epoch 1/100
756/756 [==============================] - ETA: 2:04 - loss: 2.8047 - acc: 0.164 - ETA: 30s - loss: 2.8185 - acc: 0.187 - 64s 85ms/step - loss: 2.7030 - acc: 0.1905
Epoch 2/100
756/756 [==============================] - ETA: 0s - loss: 2.1652 - acc: 0.265 - ETA: 0s - loss: 2.1870 - acc: 0.263 - 0s 469us/step - loss: 2.1269 - acc: 0.2963
Epoch 3/100
756/756 [==============================] - ETA: 0s - loss: 1.9610 - acc: 0.324 - ETA: 0s - loss: 1.8945 - acc: 0.380 - 0s 508us/step - loss: 1.8818 - acc: 0.3902
Epoch 4/100
756/756 [==============================] - ETA: 0s - loss: 1.6851 - acc: 0.410 - ETA: 0s - loss: 1.6648 - acc: 0.418 - 0s 518us/step - loss: 1.6038 - acc: 0.4484
Epoch 5/100
756/756 [==============================] - ETA: 0s - loss: 1.5342 - acc: 0.523 - ETA: 0s - loss: 1.4774 - acc: 0.521 - 0s 493us/step - loss: 1.4699 - acc: 0.5278
Epoch 6/100
756/756 [==============================] - ETA: 0s - loss: 1.3666 - acc: 0.546 - ETA: 0s - loss: 1.3726 - acc: 0.53

Epoch 48/100
756/756 [==============================] - ETA: 0s - loss: 0.7016 - acc: 0.730 - ETA: 0s - loss: 0.6524 - acc: 0.757 - 1s 688us/step - loss: 0.6641 - acc: 0.7460
Epoch 49/100
756/756 [==============================] - ETA: 0s - loss: 0.6435 - acc: 0.777 - ETA: 0s - loss: 0.6067 - acc: 0.771 - 0s 578us/step - loss: 0.6137 - acc: 0.7698
Epoch 50/100
756/756 [==============================] - ETA: 0s - loss: 0.6868 - acc: 0.742 - ETA: 0s - loss: 0.6487 - acc: 0.746 - 1s 681us/step - loss: 0.6384 - acc: 0.7474
Epoch 51/100
756/756 [==============================] - ETA: 0s - loss: 0.5711 - acc: 0.781 - ETA: 0s - loss: 0.6233 - acc: 0.761 - 0s 611us/step - loss: 0.5819 - acc: 0.7725
Epoch 52/100
756/756 [==============================] - ETA: 0s - loss: 0.6496 - acc: 0.765 - ETA: 0s - loss: 0.6301 - acc: 0.761 - 0s 653us/step - loss: 0.6284 - acc: 0.7593
Epoch 53/100
756/756 [==============================] - ETA: 0s - loss: 0.6326 - acc: 0.746 - ETA: 0s - loss: 0.6077 - acc: 0

Epoch 95/100
756/756 [==============================] - ETA: 0s - loss: 0.3696 - acc: 0.867 - ETA: 0s - loss: 0.3991 - acc: 0.851 - 0s 516us/step - loss: 0.4394 - acc: 0.8307
Epoch 96/100
756/756 [==============================] - ETA: 0s - loss: 0.5092 - acc: 0.773 - ETA: 0s - loss: 0.4908 - acc: 0.810 - 0s 518us/step - loss: 0.4527 - acc: 0.8254
Epoch 97/100
756/756 [==============================] - ETA: 0s - loss: 0.4539 - acc: 0.839 - ETA: 0s - loss: 0.5063 - acc: 0.832 - 0s 506us/step - loss: 0.4749 - acc: 0.8347
Epoch 98/100
756/756 [==============================] - ETA: 0s - loss: 0.4132 - acc: 0.847 - ETA: 0s - loss: 0.4170 - acc: 0.851 - 0s 495us/step - loss: 0.4150 - acc: 0.8492
Epoch 99/100
756/756 [==============================] - ETA: 0s - loss: 0.4348 - acc: 0.828 - ETA: 0s - loss: 0.4709 - acc: 0.804 - 0s 497us/step - loss: 0.4585 - acc: 0.8175
Epoch 100/100
756/756 [==============================] - ETA: 0s - loss: 0.4385 - acc: 0.847 - ETA: 0s - loss: 0.4433 - acc: 

In [67]:
ynew = model.predict_classes(X_test)

The accuracy will now be determined.

In [70]:
accuracy_score(y_actual2, ynew)

0.6785714285714286

In [71]:
confusion_matrix(y_actual2, ynew)

array([[ 9,  2,  0,  0,  1,  2],
       [ 1,  8,  1,  0,  0,  1],
       [ 0,  2, 10,  0,  2,  1],
       [ 0,  1,  0, 11,  0,  0],
       [ 2,  4,  0,  1,  6,  1],
       [ 3,  1,  0,  0,  1, 13]], dtype=int64)

In [72]:
print(classification_report(y_actual2, ynew))

              precision    recall  f1-score   support

           0       0.60      0.64      0.62        14
           1       0.44      0.73      0.55        11
           2       0.91      0.67      0.77        15
           3       0.92      0.92      0.92        12
           4       0.60      0.43      0.50        14
           5       0.72      0.72      0.72        18

   micro avg       0.68      0.68      0.68        84
   macro avg       0.70      0.68      0.68        84
weighted avg       0.71      0.68      0.68        84



The Matthew's correlation coefficient will now be determined.

In [73]:
matthews_corrcoef(y_actual2, ynew)

0.6182745226576499

### Two Hidden Layers

The best hyperparameters for a neural network with two hidden layers will now be determined.

In [33]:
# Function to create model, required for KerasClassifier
def create_model(optimizer='rmsprop', init='glorot_uniform', h1= 10, h2 = 10, dropout_rate=0.0, learning_rate=0.001):
    # create model
    model = Sequential()
    model.add(Dense(h1, input_dim=X_train.shape[1], kernel_initializer=init, activation='relu'))
                #activity_regularizer=regularizers.l1(0.00001)))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    model.add(Dense(h2, kernel_initializer=init, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    model.add(Dense(len(le.classes_), kernel_initializer=init, activation='softmax'))
    # Compile model
    optimizer = optimizers.Adam(lr=learning_rate,  clipvalue=100)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy']) # 
    return model

# fix random seed for reproducibility
seed = 42
np.random.seed(seed)
# create model
model = KerasClassifier(build_fn=create_model, verbose=2)
# grid search epochs, batch size and optimizer
init = ['glorot_uniform', 'normal', 'uniform']
epochs = [50, 100, 150]
batches = [64, 128, 256]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
h1 = [5, 10, 20, 40, 50, 100, 150, 200, 300, 400, 500]
h2 = [5, 10, 20, 40, 50, 100, 150, 200, 300, 400, 500]
learning_rate= [0.0001, 0.005, 0.001, 0.01, 0.05, 0.1, 0.5, 1, 5]
param_grid = dict(epochs=epochs, batch_size=batches, dropout_rate=dropout_rate, init=init, h1=h1, h2=h2, learning_rate=learning_rate)
cvi = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid, verbose=4, cv=cvi, return_train_score=True)
grid_result = grid.fit(X_train, y_actual)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] learning_rate=5, init=uniform, h2=150, h1=40, epochs=100, dropout_rate=0.2, batch_size=128 
Epoch 1/100
 - 5s - loss: 10.3940 - acc: 0.2316
Epoch 2/100
 - 0s - loss: 13.3604 - acc: 0.1711
Epoch 3/100
 - 0s - loss: 13.3604 - acc: 0.1711
Epoch 4/100
 - 0s - loss: 13.2891 - acc: 0.1755
Epoch 5/100
 - 0s - loss: 13.3842 - acc: 0.1696
Epoch 6/100
 - 0s - loss: 13.4080 - acc: 0.1681
Epoch 7/100
 - 0s - loss: 13.3842 - acc: 0.1696
Epoch 8/100
 - 0s - loss: 13.3842 - acc: 0.1696
Epoch 9/100
 - 0s - loss: 13.3604 - acc: 0.1711
Epoch 10/100
 - 0s - loss: 13.3842 - acc: 0.1696
Epoch 11/100
 - 0s - loss: 13.3604 - acc: 0.1711
Epoch 12/100
 - 0s - loss: 13.3604 - acc: 0.1711
Epoch 13/100
 - 0s - loss: 13.3129 - acc: 0.1740
Epoch 14/100
 - 0s - loss: 13.3367 - acc: 0.1726
Epoch 15/100
 - 0s - loss: 13.3842 - acc: 0.1696
Epoch 16/100
 - 0s - loss: 13.3842 - acc: 0.1696
Epoch 17/100
 - 0s - loss: 13.3129 - acc: 0.1740
Epoch 18/100
 - 0s - loss: 13.3604 - acc: 0.1711
Epoch 19/100
 - 0s - loss: 13.

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   13.4s remaining:    0.0s


[CV] learning_rate=5, init=uniform, h2=150, h1=40, epochs=100, dropout_rate=0.2, batch_size=128 
Epoch 1/100
 - 5s - loss: 10.1951 - acc: 0.2183
Epoch 2/100
 - 0s - loss: 12.3620 - acc: 0.2330
Epoch 3/100
 - 0s - loss: 12.4095 - acc: 0.2301
Epoch 4/100
 - 0s - loss: 12.3620 - acc: 0.2330
Epoch 5/100
 - 0s - loss: 12.5046 - acc: 0.2242
Epoch 6/100
 - 0s - loss: 12.3857 - acc: 0.2316
Epoch 7/100
 - 0s - loss: 12.6710 - acc: 0.2139
Epoch 8/100
 - 0s - loss: 12.3857 - acc: 0.2316
Epoch 9/100
 - 0s - loss: 12.6235 - acc: 0.2168
Epoch 10/100
 - 0s - loss: 12.5284 - acc: 0.2227
Epoch 11/100
 - 0s - loss: 12.6235 - acc: 0.2168
Epoch 12/100
 - 0s - loss: 12.5997 - acc: 0.2183
Epoch 13/100
 - 0s - loss: 12.5284 - acc: 0.2227
Epoch 14/100
 - 0s - loss: 12.3382 - acc: 0.2345
Epoch 15/100
 - 0s - loss: 12.2906 - acc: 0.2375
Epoch 16/100
 - 0s - loss: 12.4571 - acc: 0.2271
Epoch 17/100
 - 0s - loss: 12.7423 - acc: 0.2094
Epoch 18/100
 - 0s - loss: 12.2906 - acc: 0.2375
Epoch 19/100
 - 0s - loss: 12.

...

Epoch 4/150
 - 1s - loss: 1.8670 - acc: 0.2082
Epoch 5/150
 - 1s - loss: 1.7385 - acc: 0.2771
Epoch 6/150
 - 1s - loss: 1.7616 - acc: 0.2845
Epoch 7/150
 - 1s - loss: 1.7338 - acc: 0.2889
Epoch 8/150
 - 1s - loss: 1.6824 - acc: 0.3314
Epoch 9/150
 - 1s - loss: 1.6452 - acc: 0.3519
Epoch 10/150
 - 1s - loss: 1.5876 - acc: 0.3783
Epoch 11/150
 - 1s - loss: 1.5434 - acc: 0.3754
Epoch 12/150
 - 1s - loss: 1.5465 - acc: 0.3886
Epoch 13/150
 - 1s - loss: 1.5182 - acc: 0.4032
Epoch 14/150
 - 1s - loss: 1.5073 - acc: 0.4208
Epoch 15/150
 - 1s - loss: 1.5113 - acc: 0.3915
Epoch 16/150
 - 1s - loss: 1.4448 - acc: 0.4370
Epoch 17/150
 - 1s - loss: 1.4488 - acc: 0.4252
Epoch 18/150
 - 1s - loss: 1.3831 - acc: 0.4824
Epoch 19/150
 - 1s - loss: 1.4094 - acc: 0.4692
Epoch 20/150
 - 1s - loss: 1.4019 - acc: 0.4677
Epoch 21/150
 - 1s - loss: 1.3682 - acc: 0.4795
Epoch 22/150
 - 1s - loss: 1.3338 - acc: 0.5044
Epoch 23/150
 - 1s - loss: 1.3134 - acc: 0.4985
Epoch 24/150
 - 1s - loss: 1.3019 - acc: 0.516

Epoch 19/150
 - 1s - loss: 1.3951 - acc: 0.4597
Epoch 20/150
 - 1s - loss: 1.4090 - acc: 0.4656
Epoch 21/150
 - 1s - loss: 1.3823 - acc: 0.4788
Epoch 22/150
 - 1s - loss: 1.3877 - acc: 0.4700
Epoch 23/150
 - 1s - loss: 1.3828 - acc: 0.4700
Epoch 24/150
 - 1s - loss: 1.3644 - acc: 0.4802
Epoch 25/150
 - 1s - loss: 1.3279 - acc: 0.5037
Epoch 26/150
 - 1s - loss: 1.3375 - acc: 0.4641
Epoch 27/150
 - 1s - loss: 1.3174 - acc: 0.5286
Epoch 28/150
 - 1s - loss: 1.2881 - acc: 0.5212
Epoch 29/150
 - 1s - loss: 1.2923 - acc: 0.5183
Epoch 30/150
 - 1s - loss: 1.3174 - acc: 0.5022
Epoch 31/150
 - 1s - loss: 1.2857 - acc: 0.5095
Epoch 32/150
 - 1s - loss: 1.2573 - acc: 0.5242
Epoch 33/150
 - 1s - loss: 1.2777 - acc: 0.5286
Epoch 34/150
 - 1s - loss: 1.2276 - acc: 0.5388
Epoch 35/150
 - 1s - loss: 1.2565 - acc: 0.5286
Epoch 36/150
 - 1s - loss: 1.2709 - acc: 0.5095
Epoch 37/150
 - 1s - loss: 1.2391 - acc: 0.5417
Epoch 38/150
 - 1s - loss: 1.2238 - acc: 0.5520
Epoch 39/150
 - 1s - loss: 1.2274 - acc:

Epoch 34/150
 - 1s - loss: 1.2786 - acc: 0.5073
Epoch 35/150
 - 1s - loss: 1.2507 - acc: 0.5102
Epoch 36/150
 - 1s - loss: 1.2449 - acc: 0.5351
Epoch 37/150
 - 1s - loss: 1.1903 - acc: 0.5746
Epoch 38/150
 - 1s - loss: 1.2178 - acc: 0.5453
Epoch 39/150
 - 1s - loss: 1.1924 - acc: 0.5629
Epoch 40/150
 - 1s - loss: 1.1834 - acc: 0.5658
Epoch 41/150
 - 1s - loss: 1.2268 - acc: 0.5336
Epoch 42/150
 - 1s - loss: 1.1920 - acc: 0.5614
Epoch 43/150
 - 1s - loss: 1.1509 - acc: 0.5804
Epoch 44/150
 - 1s - loss: 1.1847 - acc: 0.5585
Epoch 45/150
 - 1s - loss: 1.1627 - acc: 0.5775
Epoch 46/150
 - 1s - loss: 1.1568 - acc: 0.5673
Epoch 47/150
 - 1s - loss: 1.1766 - acc: 0.5629
Epoch 48/150
 - 1s - loss: 1.1627 - acc: 0.5570
Epoch 49/150
 - 1s - loss: 1.1496 - acc: 0.5760
Epoch 50/150
 - 1s - loss: 1.1601 - acc: 0.6023
Epoch 51/150
 - 1s - loss: 1.1831 - acc: 0.5599
Epoch 52/150
 - 1s - loss: 1.1108 - acc: 0.5716
Epoch 53/150
 - 1s - loss: 1.1038 - acc: 0.5848
Epoch 54/150
 - 1s - loss: 1.1113 - acc:

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed: 63.4min finished
C:\Users\Manvir\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Epoch 1/100
 - 25s - loss: 2.3726 - acc: 0.4643
Epoch 2/100
 - 1s - loss: 2.0394 - acc: 0.5608
Epoch 3/100
 - 1s - loss: 1.5397 - acc: 0.5675
Epoch 4/100
 - 1s - loss: 1.2230 - acc: 0.6310
Epoch 5/100
 - 1s - loss: 1.2218 - acc: 0.6111
Epoch 6/100
 - 1s - loss: 1.0733 - acc: 0.6614
Epoch 7/100
 - 1s - loss: 1.0188 - acc: 0.6587
Epoch 8/100
 - 1s - loss: 0.9072 - acc: 0.6865
Epoch 9/100
 - 1s - loss: 0.9050 - acc: 0.6720
Epoch 10/100
 - 1s - loss: 0.8275 - acc: 0.6984
Epoch 11/100
 - 1s - loss: 0.7755 - acc: 0.7156
Epoch 12/100
 - 1s - loss: 0.7596 - acc: 0.7222
Epoch 13/100
 - 1s - loss: 0.7715 - acc: 0.7077
Epoch 14/100
 - 1s - loss: 0.7279 - acc: 0.7315
Epoch 15/100
 - 1s - loss: 0.7322 - acc: 0.7434
Epoch 16/100
 - 1s - loss: 0.6996 - acc: 0.7262
Epoch 17/100
 - 1s - loss: 0.7048 - acc: 0.7315
Epoch 18/100
 - 1s - loss: 0.6805 - acc: 0.7368
Epoch 19/100
 - 1s - loss: 0.7475 - acc: 0.7275
Epoch 20/100
 - 1s - loss: 0.6204 - acc: 0.7513
Epoch 21/100
 - 1s - loss: 0.6926 - acc: 0.7540


In [34]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.712963 using {'learning_rate': 0.05, 'init': 'uniform', 'h2': 300, 'h1': 300, 'epochs': 100, 'dropout_rate': 0.5, 'batch_size': 64}
0.186508 (0.047202) with: {'learning_rate': 5, 'init': 'uniform', 'h2': 150, 'h1': 40, 'epochs': 100, 'dropout_rate': 0.2, 'batch_size': 128}
0.694444 (0.073564) with: {'learning_rate': 0.1, 'init': 'normal', 'h2': 100, 'h1': 500, 'epochs': 150, 'dropout_rate': 0.4, 'batch_size': 128}
0.710317 (0.060422) with: {'learning_rate': 0.005, 'init': 'normal', 'h2': 20, 'h1': 50, 'epochs': 100, 'dropout_rate': 0.3, 'batch_size': 128}
0.697090 (0.056125) with: {'learning_rate': 0.1, 'init': 'glorot_uniform', 'h2': 200, 'h1': 40, 'epochs': 100, 'dropout_rate': 0.6, 'batch_size': 256}
0.190476 (0.036699) with: {'learning_rate': 5, 'init': 'uniform', 'h2': 150, 'h1': 200, 'epochs': 50, 'dropout_rate': 0.2, 'batch_size': 128}
0.678571 (0.045048) with: {'learning_rate': 0.0001, 'init': 'glorot_uniform', 'h2': 40, 'h1': 300, 'epochs': 50, 'dropout_rate': 0.1, 'ba

In [74]:
model = Sequential()
model.add(Dense(300, input_dim=X_train.shape[1], kernel_initializer='uniform', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(300, kernel_initializer='uniform', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(len(le.classes_), kernel_initializer='uniform', activation='softmax'))
# Compile model
optimizer = optimizers.Adam(lr=0.05,  clipvalue=100)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy']) # 

model.fit(X_train, y1, epochs = 50, batch_size = 128)

Epoch 1/50
756/756 [==============================] - ETA: 4:49 - loss: 2.0023 - acc: 0.210 - ETA: 1:55 - loss: 2.1344 - acc: 0.265 - ETA: 57s - loss: 2.6030 - acc: 0.304 - ETA: 28s - loss: 2.5280 - acc: 0.36 - ETA: 10s - loss: 2.4590 - acc: 0.38 - 60s 79ms/step - loss: 2.3698 - acc: 0.4272
Epoch 2/50
756/756 [==============================] - ETA: 1s - loss: 1.8460 - acc: 0.648 - ETA: 0s - loss: 2.2688 - acc: 0.554 - ETA: 0s - loss: 2.0195 - acc: 0.546 - ETA: 0s - loss: 2.0166 - acc: 0.537 - ETA: 0s - loss: 1.9564 - acc: 0.531 - 1s 2ms/step - loss: 1.8547 - acc: 0.5410
Epoch 3/50
756/756 [==============================] - ETA: 1s - loss: 1.7796 - acc: 0.507 - ETA: 0s - loss: 1.7144 - acc: 0.519 - ETA: 0s - loss: 1.6182 - acc: 0.500 - ETA: 0s - loss: 1.5837 - acc: 0.521 - ETA: 0s - loss: 1.4911 - acc: 0.551 - 1s 2ms/step - loss: 1.4318 - acc: 0.5648
Epoch 4/50
756/756 [==============================] - ETA: 1s - loss: 1.0860 - acc: 0.664 - ETA: 0s - loss: 1.0961 - acc: 0.660 - ETA: 0s 

In [75]:
ynew = model.predict_classes(X_test)

The accuracy will now be determined.

In [76]:
accuracy_score(y_actual2, ynew)

0.6785714285714286

In [77]:
confusion_matrix(y_actual2, ynew)

array([[ 7,  3,  0,  0,  1,  3],
       [ 0,  8,  2,  0,  0,  1],
       [ 0,  2, 10,  0,  2,  1],
       [ 0,  1,  0, 11,  0,  0],
       [ 3,  4,  0,  1,  5,  1],
       [ 1,  1,  0,  0,  0, 16]], dtype=int64)

In [78]:
print(classification_report(y_actual2, ynew))

              precision    recall  f1-score   support

           0       0.64      0.50      0.56        14
           1       0.42      0.73      0.53        11
           2       0.83      0.67      0.74        15
           3       0.92      0.92      0.92        12
           4       0.62      0.36      0.45        14
           5       0.73      0.89      0.80        18

   micro avg       0.68      0.68      0.68        84
   macro avg       0.69      0.68      0.67        84
weighted avg       0.70      0.68      0.67        84



The Matthew's correlation coefficient will now be determined.

In [79]:
matthews_corrcoef(y_actual2, ynew)

0.6201532331298863

### Three Hiden Layers

The best hyperparameters for a neural network with three hidden layers will now be determined.

In [80]:
# Function to create model, required for KerasClassifier
def create_model(optimizer='rmsprop', init='glorot_uniform', h1= 10, h2 = 10, h3=10, dropout_rate=0.0, learning_rate=0.001):
    # create model
    model = Sequential()
    model.add(Dense(h1, input_dim=X_train.shape[1], kernel_initializer=init, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    model.add(Dense(h2, kernel_initializer=init, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    model.add(Dense(h3, kernel_initializer=init, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    model.add(Dense(len(le.classes_), kernel_initializer=init, activation='softmax'))
    # Compile model
    optimizer = optimizers.Adam(lr=learning_rate,  clipvalue=100)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy']) # 
    return model

# fix random seed for reproducibility
seed = 42
np.random.seed(seed)
# create model
model = KerasClassifier(build_fn=create_model, verbose=2)
# grid search epochs, batch size and optimizer
init = ['glorot_uniform', 'normal', 'uniform']
epochs = [50, 100, 150]
batches = [64, 128, 256]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
h1 = [5, 10, 20, 40, 50, 100, 150, 200, 300, 400, 500]
h2 = [5, 10, 20, 40, 50, 100, 150, 200, 300, 400, 500]
h3 = [5, 10, 20, 40, 50, 100, 150, 200, 300, 400, 500]
learning_rate= [0.0001, 0.005, 0.001, 0.01, 0.05, 0.1, 0.5, 1, 5]
param_grid = dict(epochs=epochs, batch_size=batches, dropout_rate=dropout_rate, init=init, h1=h1, h2=h2, h3=h3, learning_rate=learning_rate)
cvi = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid, verbose=4, cv=cvi, return_train_score=True)
grid_result = grid.fit(X_train, y_actual)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] learning_rate=1, init=glorot_uniform, h3=300, h2=20, h1=300, epochs=100, dropout_rate=0.0, batch_size=256 
Epoch 1/100
 - 62s - loss: 8.5203 - acc: 0.1770
Epoch 2/100
 - 0s - loss: 11.0658 - acc: 0.3127
Epoch 3/100
 - 0s - loss: 11.5165 - acc: 0.2847
Epoch 4/100
 - 0s - loss: 12.1956 - acc: 0.2434
Epoch 5/100
 - 0s - loss: 13.0514 - acc: 0.1903
Epoch 6/100
 - 0s - loss: 13.2653 - acc: 0.1770
Epoch 7/100
 - 0s - loss: 13.4080 - acc: 0.1681
Epoch 8/100
 - 0s - loss: 13.4555 - acc: 0.1652
Epoch 9/100
 - 0s - loss: 13.4555 - acc: 0.1652
Epoch 10/100
 - 0s - loss: 13.4793 - acc: 0.1637
Epoch 11/100
 - 0s - loss: 13.4793 - acc: 0.1637
Epoch 12/100
 - 0s - loss: 13.5268 - acc: 0.1608
Epoch 13/100
 - 0s - loss: 13.5268 - acc: 0.1608
Epoch 14/100
 - 0s - loss: 13.5031 - acc: 0.1622
Epoch 15/100
 - 0s - loss: 13.5268 - acc: 0.1608
Epoch 16/100
 - 0s - loss: 13.5031 - acc: 0.1622
Epoch 17/100
 - 0s - loss: 13.5268 - acc: 0.1608
Epoch 18/100
 - 0s - loss: 13.5268 - acc: 0.1608
Epoch 19/100
 -

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.1min remaining:    0.0s


[CV] learning_rate=1, init=glorot_uniform, h3=300, h2=20, h1=300, epochs=100, dropout_rate=0.0, batch_size=256 
Epoch 1/100
 - 61s - loss: 8.7256 - acc: 0.1549
Epoch 2/100
 - 0s - loss: 11.2299 - acc: 0.3009
Epoch 3/100
 - 0s - loss: 11.1471 - acc: 0.3068
Epoch 4/100
 - 0s - loss: 12.0529 - acc: 0.2522
Epoch 5/100
 - 0s - loss: 12.1247 - acc: 0.2478
Epoch 6/100
 - 0s - loss: 11.9103 - acc: 0.2611
Epoch 7/100
 - 0s - loss: 11.8865 - acc: 0.2625
Epoch 8/100
 - 0s - loss: 11.6531 - acc: 0.2758
Epoch 9/100
 - 0s - loss: 11.6488 - acc: 0.2773
Epoch 10/100
 - 0s - loss: 12.5293 - acc: 0.2227
Epoch 11/100
 - 0s - loss: 12.0730 - acc: 0.2507
Epoch 12/100
 - 0s - loss: 11.2446 - acc: 0.3024
Epoch 13/100
 - 0s - loss: 10.8880 - acc: 0.3245
Epoch 14/100
 - 0s - loss: 10.8880 - acc: 0.3245
Epoch 15/100
 - 0s - loss: 10.7873 - acc: 0.3304
Epoch 16/100
 - 0s - loss: 11.6488 - acc: 0.2773
Epoch 17/100
 - 0s - loss: 12.0291 - acc: 0.2537
Epoch 18/100
 - 0s - loss: 12.2669 - acc: 0.2389
Epoch 19/100
 -

...

Epoch 79/150
 - 1s - loss: 13.4239 - acc: 0.1672
Epoch 80/150
 - 1s - loss: 13.5657 - acc: 0.1584
Epoch 81/150
 - 1s - loss: 13.4711 - acc: 0.1642
Epoch 82/150
 - 1s - loss: 13.2584 - acc: 0.1774
Epoch 83/150
 - 1s - loss: 13.3293 - acc: 0.1730
Epoch 84/150
 - 1s - loss: 13.4475 - acc: 0.1657
Epoch 85/150
 - 1s - loss: 13.3766 - acc: 0.1701
Epoch 86/150
 - 1s - loss: 13.3530 - acc: 0.1716
Epoch 87/150
 - 1s - loss: 13.3766 - acc: 0.1701
Epoch 88/150
 - 1s - loss: 13.5184 - acc: 0.1613
Epoch 89/150
 - 1s - loss: 13.4475 - acc: 0.1657
Epoch 90/150
 - 1s - loss: 13.6838 - acc: 0.1510
Epoch 91/150
 - 1s - loss: 13.2821 - acc: 0.1760
Epoch 92/150
 - 1s - loss: 13.3766 - acc: 0.1701
Epoch 93/150
 - 1s - loss: 13.4776 - acc: 0.1628
Epoch 94/150
 - 1s - loss: 13.2112 - acc: 0.1804
Epoch 95/150
 - 1s - loss: 13.3293 - acc: 0.1730
Epoch 96/150
 - 1s - loss: 13.3057 - acc: 0.1745
Epoch 97/150
 - 1s - loss: 13.5420 - acc: 0.1598
Epoch 98/150
 - 1s - loss: 13.5048 - acc: 0.1613
Epoch 99/150
 - 1s -

Epoch 90/150
 - 1s - loss: 13.4042 - acc: 0.1684
Epoch 91/150
 - 1s - loss: 13.4514 - acc: 0.1654
Epoch 92/150
 - 1s - loss: 13.4750 - acc: 0.1640
Epoch 93/150
 - 1s - loss: 13.3806 - acc: 0.1698
Epoch 94/150
 - 1s - loss: 13.4278 - acc: 0.1669
Epoch 95/150
 - 1s - loss: 13.4278 - acc: 0.1669
Epoch 96/150
 - 1s - loss: 13.4042 - acc: 0.1684
Epoch 97/150
 - 1s - loss: 13.4278 - acc: 0.1669
Epoch 98/150
 - 1s - loss: 13.4042 - acc: 0.1684
Epoch 99/150
 - 1s - loss: 13.4042 - acc: 0.1684
Epoch 100/150
 - 1s - loss: 13.4750 - acc: 0.1640
Epoch 101/150
 - 1s - loss: 13.4514 - acc: 0.1654
Epoch 102/150
 - 1s - loss: 13.4042 - acc: 0.1684
Epoch 103/150
 - 1s - loss: 13.4750 - acc: 0.1640
Epoch 104/150
 - 1s - loss: 13.4042 - acc: 0.1684
Epoch 105/150
 - 1s - loss: 13.4278 - acc: 0.1669
Epoch 106/150
 - 1s - loss: 13.3570 - acc: 0.1713
Epoch 107/150
 - 1s - loss: 13.2862 - acc: 0.1757
Epoch 108/150
 - 1s - loss: 13.4278 - acc: 0.1669
Epoch 109/150
 - 1s - loss: 13.5222 - acc: 0.1611
Epoch 110/

Epoch 101/150
 - 1s - loss: 13.5260 - acc: 0.1608
Epoch 102/150
 - 1s - loss: 13.5024 - acc: 0.1623
Epoch 103/150
 - 1s - loss: 13.5260 - acc: 0.1608
Epoch 104/150
 - 1s - loss: 13.5496 - acc: 0.1594
Epoch 105/150
 - 1s - loss: 13.5260 - acc: 0.1608
Epoch 106/150
 - 1s - loss: 13.5260 - acc: 0.1608
Epoch 107/150
 - 1s - loss: 13.5260 - acc: 0.1608
Epoch 108/150
 - 1s - loss: 13.5731 - acc: 0.1579
Epoch 109/150
 - 1s - loss: 13.5024 - acc: 0.1623
Epoch 110/150
 - 1s - loss: 13.5260 - acc: 0.1608
Epoch 111/150
 - 1s - loss: 13.5496 - acc: 0.1594
Epoch 112/150
 - 1s - loss: 13.5260 - acc: 0.1608
Epoch 113/150
 - 1s - loss: 13.5496 - acc: 0.1594
Epoch 114/150
 - 1s - loss: 13.5260 - acc: 0.1608
Epoch 115/150
 - 1s - loss: 13.5260 - acc: 0.1608
Epoch 116/150
 - 1s - loss: 13.5731 - acc: 0.1579
Epoch 117/150
 - 1s - loss: 13.5260 - acc: 0.1608
Epoch 118/150
 - 1s - loss: 13.5496 - acc: 0.1594
Epoch 119/150
 - 1s - loss: 13.5731 - acc: 0.1579
Epoch 120/150
 - 1s - loss: 13.5496 - acc: 0.1594


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed: 303.9min finished
C:\Users\Manvir\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Epoch 1/150
 - 138s - loss: 2.7113 - acc: 0.3042
Epoch 2/150
 - 1s - loss: 2.0137 - acc: 0.4709
Epoch 3/150
 - 1s - loss: 1.5086 - acc: 0.5648
Epoch 4/150
 - 1s - loss: 1.2074 - acc: 0.5780
Epoch 5/150
 - 1s - loss: 1.0395 - acc: 0.5489
Epoch 6/150
 - 1s - loss: 0.9998 - acc: 0.6124
Epoch 7/150
 - 1s - loss: 0.9222 - acc: 0.6429
Epoch 8/150
 - 1s - loss: 0.9146 - acc: 0.6362
Epoch 9/150
 - 1s - loss: 0.8297 - acc: 0.6733
Epoch 10/150
 - 1s - loss: 0.7887 - acc: 0.6667
Epoch 11/150
 - 1s - loss: 0.8095 - acc: 0.6905
Epoch 12/150
 - 1s - loss: 0.7797 - acc: 0.6786
Epoch 13/150
 - 1s - loss: 0.7416 - acc: 0.7130
Epoch 14/150
 - 1s - loss: 0.7333 - acc: 0.7130
Epoch 15/150
 - 1s - loss: 0.6715 - acc: 0.7288
Epoch 16/150
 - 1s - loss: 0.6908 - acc: 0.7288
Epoch 17/150
 - 1s - loss: 0.6600 - acc: 0.7407
Epoch 18/150
 - 1s - loss: 0.6614 - acc: 0.7474
Epoch 19/150
 - 1s - loss: 0.6387 - acc: 0.7500
Epoch 20/150
 - 1s - loss: 0.6162 - acc: 0.7619
Epoch 21/150
 - 1s - loss: 0.5927 - acc: 0.7659

In [81]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.702381 using {'learning_rate': 0.05, 'init': 'normal', 'h3': 300, 'h2': 20, 'h1': 150, 'epochs': 150, 'dropout_rate': 0.1, 'batch_size': 256}
0.205026 (0.050568) with: {'learning_rate': 1, 'init': 'glorot_uniform', 'h3': 300, 'h2': 20, 'h1': 300, 'epochs': 100, 'dropout_rate': 0.0, 'batch_size': 256}
0.174603 (0.020499) with: {'learning_rate': 0.5, 'init': 'uniform', 'h3': 400, 'h2': 40, 'h1': 5, 'epochs': 150, 'dropout_rate': 0.5, 'batch_size': 256}
0.641534 (0.042196) with: {'learning_rate': 0.0001, 'init': 'uniform', 'h3': 40, 'h2': 40, 'h1': 5, 'epochs': 150, 'dropout_rate': 0.0, 'batch_size': 256}
0.702381 (0.054813) with: {'learning_rate': 0.05, 'init': 'normal', 'h3': 300, 'h2': 20, 'h1': 150, 'epochs': 150, 'dropout_rate': 0.1, 'batch_size': 256}
0.578042 (0.065149) with: {'learning_rate': 0.1, 'init': 'glorot_uniform', 'h3': 100, 'h2': 10, 'h1': 10, 'epochs': 150, 'dropout_rate': 0.5, 'batch_size': 128}
0.492063 (0.040251) with: {'learning_rate': 0.0001, 'init': 'gloro

In [30]:
model = Sequential()
model.add(Dense(150, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Dense(20, kernel_initializer='normal', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Dense(300, kernel_initializer='normal', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Dense(len(le.classes_), kernel_initializer='normal', activation='softmax'))
# Compile model
optimizer = optimizers.Adam(lr=0.05,  clipvalue=100)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

model.fit(X_train, y1, epochs = 150, batch_size = 256)

Epoch 1/150
756/756 [==============================] - ETA: 1s - loss: 1.9359 - acc: 0.246 - 1s 1ms/step - loss: 2.5668 - acc: 0.4114
Epoch 2/150
756/756 [==============================] - ETA: 0s - loss: 1.6262 - acc: 0.503 - 0s 44us/step - loss: 1.7148 - acc: 0.5397
Epoch 3/150
756/756 [==============================] - ETA: 0s - loss: 1.2996 - acc: 0.597 - 0s 40us/step - loss: 1.3042 - acc: 0.5939
Epoch 4/150
756/756 [==============================] - ETA: 0s - loss: 1.1195 - acc: 0.664 - 0s 42us/step - loss: 1.0929 - acc: 0.6561
Epoch 5/150
756/756 [==============================] - ETA: 0s - loss: 0.9009 - acc: 0.656 - 0s 37us/step - loss: 1.0671 - acc: 0.6323
Epoch 6/150
756/756 [==============================] - ETA: 0s - loss: 0.8524 - acc: 0.687 - 0s 42us/step - loss: 0.9316 - acc: 0.6786
Epoch 7/150
756/756 [==============================] - ETA: 0s - loss: 1.0319 - acc: 0.644 - 0s 40us/step - loss: 0.9593 - acc: 0.6746
Epoch 8/150
756/756 [==============================] - E

756/756 [==============================] - ETA: 0s - loss: 0.0849 - acc: 0.980 - 0s 37us/step - loss: 0.1141 - acc: 0.9669
Epoch 122/150
756/756 [==============================] - ETA: 0s - loss: 0.0808 - acc: 0.968 - 0s 40us/step - loss: 0.0940 - acc: 0.9643
Epoch 123/150
756/756 [==============================] - ETA: 0s - loss: 0.0978 - acc: 0.968 - 0s 37us/step - loss: 0.0963 - acc: 0.9656
Epoch 124/150
756/756 [==============================] - ETA: 0s - loss: 0.1088 - acc: 0.964 - 0s 41us/step - loss: 0.1051 - acc: 0.9643
Epoch 125/150
756/756 [==============================] - ETA: 0s - loss: 0.1336 - acc: 0.945 - 0s 41us/step - loss: 0.1214 - acc: 0.9484
Epoch 126/150
756/756 [==============================] - ETA: 0s - loss: 0.0667 - acc: 0.984 - 0s 34us/step - loss: 0.1078 - acc: 0.9683
Epoch 127/150
756/756 [==============================] - ETA: 0s - loss: 0.1557 - acc: 0.933 - 0s 33us/step - loss: 0.1415 - acc: 0.9524
Epoch 128/150
756/756 [==============================] 

In [31]:
ynew = model.predict_classes(X_test)

The accuracy will now be determined.

In [32]:
accuracy_score(y_actual2, ynew)

0.6547619047619048

In [33]:
confusion_matrix(y_actual2, ynew)

array([[11,  1,  0,  0,  0,  2],
       [ 1,  8,  1,  0,  0,  1],
       [ 2,  0,  9,  1,  2,  1],
       [ 0,  1,  0, 11,  0,  0],
       [ 5,  4,  0,  1,  3,  1],
       [ 3,  2,  0,  0,  0, 13]], dtype=int64)

In [34]:
print(classification_report(y_actual2, ynew))

              precision    recall  f1-score   support

           0       0.50      0.79      0.61        14
           1       0.50      0.73      0.59        11
           2       0.90      0.60      0.72        15
           3       0.85      0.92      0.88        12
           4       0.60      0.21      0.32        14
           5       0.72      0.72      0.72        18

   micro avg       0.65      0.65      0.65        84
   macro avg       0.68      0.66      0.64        84
weighted avg       0.69      0.65      0.64        84



The Matthew's correlation coefficient will now be determined.

In [35]:
matthews_corrcoef(y_actual2, ynew)

0.5951329550472182